# Homework 4 - MILP
### Network Design

First we import the necessary variables

In [85]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

Setting our variables and the data provided

In [86]:
# Data

Regions = ['East','South','Midwest','West']

Demand = dict(zip(Regions,[100_000,150_000,110_000,90_000]))

Sites = ['Atlanta','Chicago','NewYork','SanDiego']

Capacity_option = ['150k','300k']



In [87]:
Demand

{'East': 100000, 'South': 150000, 'Midwest': 110000, 'West': 90000}

In [88]:
#Variables

Routes = list(product(Sites,Regions))
Routes


[('Atlanta', 'East'),
 ('Atlanta', 'South'),
 ('Atlanta', 'Midwest'),
 ('Atlanta', 'West'),
 ('Chicago', 'East'),
 ('Chicago', 'South'),
 ('Chicago', 'Midwest'),
 ('Chicago', 'West'),
 ('NewYork', 'East'),
 ('NewYork', 'South'),
 ('NewYork', 'Midwest'),
 ('NewYork', 'West'),
 ('SanDiego', 'East'),
 ('SanDiego', 'South'),
 ('SanDiego', 'Midwest'),
 ('SanDiego', 'West')]

In [89]:
Capacity_plant_option = list(product(Sites,Capacity_option))

Capacity_plant_option

[('Atlanta', '150k'),
 ('Atlanta', '300k'),
 ('Chicago', '150k'),
 ('Chicago', '300k'),
 ('NewYork', '150k'),
 ('NewYork', '300k'),
 ('SanDiego', '150k'),
 ('SanDiego', '300k')]

In [90]:
Capacity_plant_value = dict(zip(Capacity_plant_option,[150_000,300_000]*4))
Capacity_plant_value

{('Atlanta', '150k'): 150000,
 ('Atlanta', '300k'): 300000,
 ('Chicago', '150k'): 150000,
 ('Chicago', '300k'): 300000,
 ('NewYork', '150k'): 150000,
 ('NewYork', '300k'): 300000,
 ('SanDiego', '150k'): 150000,
 ('SanDiego', '300k'): 300000}

In [91]:
Var_costs_list = [211,232,240,300,
              232,212,230,280,
              238,230,215,270,
              299,280,270,225]

VarCost=dict(zip(Routes,Var_costs_list))

VarCost

{('Atlanta', 'East'): 211,
 ('Atlanta', 'South'): 232,
 ('Atlanta', 'Midwest'): 240,
 ('Atlanta', 'West'): 300,
 ('Chicago', 'East'): 232,
 ('Chicago', 'South'): 212,
 ('Chicago', 'Midwest'): 230,
 ('Chicago', 'West'): 280,
 ('NewYork', 'East'): 238,
 ('NewYork', 'South'): 230,
 ('NewYork', 'Midwest'): 215,
 ('NewYork', 'West'): 270,
 ('SanDiego', 'East'): 299,
 ('SanDiego', 'South'): 280,
 ('SanDiego', 'Midwest'): 270,
 ('SanDiego', 'West'): 225}

In [92]:
Fix_costs_list = [i*1_000_000 for i in [4.5,7,5,8,5.5,8.5,6,10]]

FixCost = dict(zip(Capacity_plant_option,Fix_costs_list))

FixCost


{('Atlanta', '150k'): 4500000.0,
 ('Atlanta', '300k'): 7000000,
 ('Chicago', '150k'): 5000000,
 ('Chicago', '300k'): 8000000,
 ('NewYork', '150k'): 5500000.0,
 ('NewYork', '300k'): 8500000.0,
 ('SanDiego', '150k'): 6000000,
 ('SanDiego', '300k'): 10000000}

### Defining the Model

In [93]:
m = gp.Model('NetworkDesign_Team:AnacondaArtillery')

### Decision Variables

In [94]:
#Routes variable: From City to Region | Integer
Routes_ij = m.addVars(Routes, vtype=GRB.INTEGER, name='Routes')
Routes_ij

{('Atlanta', 'East'): <gurobi.Var *Awaiting Model Update*>,
 ('Atlanta', 'South'): <gurobi.Var *Awaiting Model Update*>,
 ('Atlanta', 'Midwest'): <gurobi.Var *Awaiting Model Update*>,
 ('Atlanta', 'West'): <gurobi.Var *Awaiting Model Update*>,
 ('Chicago', 'East'): <gurobi.Var *Awaiting Model Update*>,
 ('Chicago', 'South'): <gurobi.Var *Awaiting Model Update*>,
 ('Chicago', 'Midwest'): <gurobi.Var *Awaiting Model Update*>,
 ('Chicago', 'West'): <gurobi.Var *Awaiting Model Update*>,
 ('NewYork', 'East'): <gurobi.Var *Awaiting Model Update*>,
 ('NewYork', 'South'): <gurobi.Var *Awaiting Model Update*>,
 ('NewYork', 'Midwest'): <gurobi.Var *Awaiting Model Update*>,
 ('NewYork', 'West'): <gurobi.Var *Awaiting Model Update*>,
 ('SanDiego', 'East'): <gurobi.Var *Awaiting Model Update*>,
 ('SanDiego', 'South'): <gurobi.Var *Awaiting Model Update*>,
 ('SanDiego', 'Midwest'): <gurobi.Var *Awaiting Model Update*>,
 ('SanDiego', 'West'): <gurobi.Var *Awaiting Model Update*>}

In [95]:
#Capacity variable: For City which Capacity (150k or 300k) | Binary
Cap_plant_ij = m.addVars(Capacity_plant_option, vtype=GRB.BINARY,name='Capacity_type')
Cap_plant_ij

{('Atlanta', '150k'): <gurobi.Var *Awaiting Model Update*>,
 ('Atlanta', '300k'): <gurobi.Var *Awaiting Model Update*>,
 ('Chicago', '150k'): <gurobi.Var *Awaiting Model Update*>,
 ('Chicago', '300k'): <gurobi.Var *Awaiting Model Update*>,
 ('NewYork', '150k'): <gurobi.Var *Awaiting Model Update*>,
 ('NewYork', '300k'): <gurobi.Var *Awaiting Model Update*>,
 ('SanDiego', '150k'): <gurobi.Var *Awaiting Model Update*>,
 ('SanDiego', '300k'): <gurobi.Var *Awaiting Model Update*>}

### Objective Function

In [96]:
#Objective Function: Total Cost = Variable Cost + Fixed Cost

m.setObjective(Routes_ij.prod(VarCost) + Cap_plant_ij.prod(FixCost), GRB.MINIMIZE) 

### Constraints

In [97]:
#Demand Constraints
m.addConstrs((Routes_ij.sum('*',r)>=Demand[r] for r in Regions), name='Demand')

{'East': <gurobi.Constr *Awaiting Model Update*>,
 'South': <gurobi.Constr *Awaiting Model Update*>,
 'Midwest': <gurobi.Constr *Awaiting Model Update*>,
 'West': <gurobi.Constr *Awaiting Model Update*>}

In [98]:
#Capacity Constraints

m.addConstrs((Routes_ij.sum(s)-Cap_plant_ij.prod(Capacity_plant_value, s)<=0 for s in Sites), name='Capacity')

{'Atlanta': <gurobi.Constr *Awaiting Model Update*>,
 'Chicago': <gurobi.Constr *Awaiting Model Update*>,
 'NewYork': <gurobi.Constr *Awaiting Model Update*>,
 'SanDiego': <gurobi.Constr *Awaiting Model Update*>}

In [99]:
#Select one capacity per plant
m.addConstrs((Cap_plant_ij.sum(s) <= 1 for s in Sites), name = 'Only1Capacity')

{'Atlanta': <gurobi.Constr *Awaiting Model Update*>,
 'Chicago': <gurobi.Constr *Awaiting Model Update*>,
 'NewYork': <gurobi.Constr *Awaiting Model Update*>,
 'SanDiego': <gurobi.Constr *Awaiting Model Update*>}

In [113]:
#Select at least one plant
m.addConstr(Cap_plant_ij.sum('*','*') >= 1, name='AtLeast1Site')

<gurobi.Constr *Awaiting Model Update*>

In [101]:
#linking Constraints
m.addConstrs((Routes_ij.sum(s) - Cap_plant_ij.sum(s)*10_000_000 <=0 for s in Sites), name='Linking')

{'Atlanta': <gurobi.Constr *Awaiting Model Update*>,
 'Chicago': <gurobi.Constr *Awaiting Model Update*>,
 'NewYork': <gurobi.Constr *Awaiting Model Update*>,
 'SanDiego': <gurobi.Constr *Awaiting Model Update*>}

### Writting the model

In [102]:
m.write('NetworkDesign.lp')

In [112]:
# Details of the model
with open('NetworkDesign.lp') as f:
  print (f.read())

\ Model NetworkDesign_Team:AnacondaArtillery
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  211 Routes[Atlanta,East] + 232 Routes[Atlanta,South]
   + 240 Routes[Atlanta,Midwest] + 300 Routes[Atlanta,West]
   + 232 Routes[Chicago,East] + 212 Routes[Chicago,South]
   + 230 Routes[Chicago,Midwest] + 280 Routes[Chicago,West]
   + 238 Routes[NewYork,East] + 230 Routes[NewYork,South]
   + 215 Routes[NewYork,Midwest] + 270 Routes[NewYork,West]
   + 299 Routes[SanDiego,East] + 280 Routes[SanDiego,South]
   + 270 Routes[SanDiego,Midwest] + 225 Routes[SanDiego,West]
   + 4.5e+06 Capacity_type[Atlanta,150k]
   + 7e+06 Capacity_type[Atlanta,300k] + 5e+06 Capacity_type[Chicago,150k]
   + 8e+06 Capacity_type[Chicago,300k]
   + 5.5e+06 Capacity_type[NewYork,150k]
   + 8.5e+06 Capacity_type[NewYork,300k]
   + 6e+06 Capacity_type[SanDiego,150k]
   + 1e+07 Capacity_type[SanDiego,300k]
Subject To
 Demand[East]: Routes[Atlanta,East] + Routes[Chicago,East]
   + Ro

### Optimize Model

In [103]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 24 columns and 80 nonzeros
Model fingerprint: 0x462de2e0
Variable types: 0 continuous, 24 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+02, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 1.195500e+08
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 13 rows, 24 columns, 56 nonzeros
Variable types: 0 continuous, 24 integer (8 binary)

Root relaxation: objective 1.092500e+08, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.0925e+08    0    4 1.

### Optimization Result

In [111]:
print(f"Optimal objective value: {m.objVal}")

print("\nSites Openned",end = '\n')
for site, capacity in Cap_plant_ij:
  if (Cap_plant_ij[site,capacity].x>0):
    print(f'Open Site in {site} with capacity: {capacity}')

print("\nShipment plan:")
for site, region in Routes_ij.keys():
    if (abs(Routes_ij[site, region].x) > 0): #Only print if not 0
        print (f"Ship {Routes_ij[site, region].x} units to region {region} from site {site}")
        


Optimal objective value: 116400000.0

Sites Openned
Open Site in Atlanta with capacity: 150k
Open Site in Chicago with capacity: 300k

Shipment plan:
Ship 100000.0 units to region East from site Atlanta
Ship 50000.0 units to region Midwest from site Atlanta
Ship 150000.0 units to region South from site Chicago
Ship 60000.0 units to region Midwest from site Chicago
Ship 90000.0 units to region West from site Chicago
